<a href="https://colab.research.google.com/github/bckim9489/agent_practice/blob/main/agent_pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Common Setup

참고 : 아래 Secret으로 Key 등록해서 실습함
1.   LLM API Key (GPT, claude 등) : OPENAI-KEY 로 등록했음
2.   Tavily Serach API Key (https://app.tavily.com/home) : TAVIL-KEY 로 등록했음



In [ ]:
!pip -q install -U langgraph langchain-openai langchain-community wikipedia numexpr tavily-python httpx==0.27.2

In [ ]:
import os
from langchain_openai import ChatOpenAI
from google.colab import userdata

# LLM API Key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI-KEY")

# Tools
# Search API Key
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY-KEY")

llm = ChatOpenAI(model="gpt-4o", temperature=0)


## ㄴ 0.1 Tools

In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_core.tools import tool
import numexpr

# Wikipedia tool
wiki = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=2000)
)

# 기존 wiki wrapper 재사용
wiki_lookup_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=4000)

@tool
def docstore_lookup(title: str) -> str:
    """
    Lookup a specific document by title from the docstore (Wikipedia).
    Input should be a precise title or entity name.
    """
    return wiki_lookup_wrapper.run(title)

# Tavily Search tool
search_tool = TavilySearchResults(max_results=5)

# Calculator tool (llm-math 대체)
@tool
def calculator(expression: str) -> str:
    """
    Evaluate a mathematical expression.
    Input must be a valid expression like '345*872' or '12/4+9'.
    """
    try:
        return str(numexpr.evaluate(expression))
    except Exception as e:
        return f"Error: {e}"


tools = [wiki, calculator, search_tool, docstore_lookup]

/tmp/ipython-input-4054356365.py:25: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=5)


# 1. Zero-Shot ReAct
*   기억 없음(Stateless)
*   **LLM + Tools**





## ㄴ 1.1 Description

### ㄴ 1.1.1 Workflow


```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph Runtime 시작
  ↓
LLM: 질문 해석 (Thought / Reasoning)
  ↓
Tool 필요 여부 판단
  ├─ 필요 없음 → 바로 답 생성
  └─ 필요 있음 → Tool 선택 (Action)
                      ↓
                Tool 실행 (Python / API / DB 등)
                      ↓
                결과 반환 (Observation)
                      ↓
LLM이 결과 반영하여 다시 Reasoning
  ↓
(필요 시 Thought → Action → Observation 반복)
  ↓
Final Answer 생성
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시

```

### ㄴ 1.1.2 Inner ReAct Loop



``` javascript
while 문제 해결 전:
    Thought      → 어떻게 풀지 스스로 판단
    Action       → 사용할 Tool 선택
    Observation  → Tool 실행 결과 받기
    Thought      → 결과 보고 다음 행동 결정
```



## ㄴ 1.2 Code

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

/tmp/ipython-input-2357398768.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


In [ ]:
q = """위키피디아에서 찾기 좋은 '검색어(제목형 키워드)'를 3개 만들어서,
그 키워드로 Wikipedia tool을 사용해 검색하고 근거를 요약해줘.
주제: Pinus densiflora 옮겨심기(이식) 적기"""

In [ ]:
result = agent.invoke({"messages": [("user", q)]})

print(result["messages"][-1].content)

위키피디아에서 "Pinus densiflora transplanting season", "best time to transplant Japanese red pine", "Pinus densiflora planting season"에 대한 검색 결과는 유용한 정보를 제공하지 않았습니다. 이 주제에 대한 구체적인 정보는 위키피디아에 없을 수 있습니다. 대신, 관련된 원예 서적이나 전문 웹사이트에서 더 많은 정보를 찾을 수 있을 것입니다.


## ㄴ 1.3 Verbose

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

for step in agent.stream(
    {"messages": [("user", q)]},
    stream_mode="values",
):
    msg = step["messages"][-1]

    # LLM이 생각/결정한 내용
    if isinstance(msg, AIMessage):
        print("\n AI MESSAGE")
        print(msg.content)

        if getattr(msg, "tool_calls", None):
            print(" TOOL CALL:", msg.tool_calls)

    # Tool 실행 결과
    elif isinstance(msg, ToolMessage):
        print("\n TOOL RESULT")
        print("tool:", msg.name)
        print(msg.content[:1000])  # 너무 길면 앞부분만



🧠 AI MESSAGE

🔧 TOOL CALL: [{'name': 'wikipedia', 'args': {'query': 'Pinus densiflora transplanting season'}, 'id': 'call_YsQdtSI1bGFqqt0jgag3Wbkz', 'type': 'tool_call'}, {'name': 'wikipedia', 'args': {'query': 'best time to transplant Japanese red pine'}, 'id': 'call_5JvpdLu9Y97nDSxqNif0DOyk', 'type': 'tool_call'}, {'name': 'wikipedia', 'args': {'query': 'Pinus densiflora cultivation'}, 'id': 'call_50lYIe0IKFddirGW9cMlnUfc', 'type': 'tool_call'}]

📦 TOOL RESULT
tool: wikipedia
Page: Pinus sylvestris
Summary: Pinus sylvestris, the Scots pine (UK), Scotch pine (US), Baltic pine, or European red pine is a species of tree in the pine family Pinaceae that is native to Eurasia. It can readily be identified by its combination of fairly short, blue-green leaves and orange-red bark.

Page: Lactarius hatsudake
Summary: Lactarius hatsudake, also known as red milk mushroom, is a species of agaric fungus in the family Russulaceae native to Asia, first described by Tanaka Nobujirō in 1890. It is a

# 2. Conversational ReAct
*   메모리 사용
*   **LLM + Tools + Checkpointer**



## ㄴ 2.1 Description

### ㄴ 2.1.1 Workflow

```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph Runtime 시작
  ↓
이전 대화 기록(Memory / State) 로드
  ↓
LLM: 현재 질문 + 대화 히스토리 함께 해석
  ↓
사용자의 의도 파악 (맥락 기반 Reasoning)
  ↓
Tool 필요 여부 판단
  ├─ 필요 없음 → 바로 답 생성
  └─ 필요 있음 → Tool 선택 (Action)
                      ↓
                Tool 실행 (Python / API / DB 등)
                      ↓
                결과 반환 (Observation)
                      ↓
LLM이 결과 + 이전 대화 맥락을 반영하여 다시 Reasoning
  ↓
(필요 시 Thought → Action → Observation 반복)
  ↓
Final Answer 생성
  ↓
대화 Memory(State)에 이번 메시지 저장
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시
```

### ㄴ 2.1.2 Inner ReAct Loop

```javascript
while 문제 해결 전:
    Load Memory        → 이전 대화 불러오기
    Thought            → 맥락 기반으로 판단
    Action             → Tool 필요하면 실행
    Observation        → Tool 결과 받기
    Update Memory      → 새 정보 저장
```

## ㄴ 2.2 Code

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()
agent = create_react_agent(llm, tools, checkpointer=checkpointer)

/tmp/ipython-input-1647517424.py:5: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools, checkpointer=checkpointer)


In [ ]:
config = {"configurable": {"thread_id": "chat-001"}}
q1 = "내 출생년도는 1994년이야. 지금은 계산하지 말고 이 정보만 기억해."
q2 = "그럼 내가 2026년에 몇 살인지 계산기를 꼭 사용해서 나이를 계산해서 알려줘"
q3 = "올해는 무슨 해이며 나는 무슨 띠인지 알려줘"

In [ ]:
result = agent.invoke({"messages": [("user", q1)]}, config=config)
print(result["messages"][-1].content)
result = agent.invoke({"messages": [("user", q2)]}, config=config)
print(result["messages"][-1].content)
result = agent.invoke({"messages": [("user", q3)]}, config=config)
print(result["messages"][-1].content)


알겠습니다. 당신의 출생년도는 1994년입니다. 이 정보를 기억하겠습니다.
2026년에 당신은 32살이 됩니다.
2023년은 토끼띠의 해입니다. 당신은 1994년생이므로 개띠입니다.


## ㄴ 2.3 Verbose

### ㄴ 2.3.1 Test Setup

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

def run_verbose(question: str, label: str, thread_id: str):
    print("\n" + "="*20 + f" {label} (thread_id={thread_id}) " + "="*20)
    print("USER:", question)

    cfg = {"configurable": {"thread_id": thread_id}}

    for step in agent.stream(
        {"messages": [("user", question)]},
        config=cfg,
        stream_mode="values",
    ):
        msg = step["messages"][-1]

        if isinstance(msg, AIMessage):
            if msg.content:
                print("\nAI:", msg.content)
            if getattr(msg, "tool_calls", None):
                print("tool_calls:", msg.tool_calls)

        elif isinstance(msg, ToolMessage):
            print("\nTOOL RESULT:", msg.name)
            print(msg.content[:800])

#메모리 사용하는지 A/B 테스트
def ab_test(q1: str, q2: str, q3:str, thread_a: str="mem-A", thread_b: str="mem-B", thread_c: str="mem-C"):
    print("\n\n" + "#"*10 + " A/B MEMORY TEST START " + "#"*10)

    # A: q1 using thread_id a
    run_verbose(q1, "A-1: seed memory (q1)", thread_a)


    # B: q2 using thread_id b
    run_verbose(q2, "B-1: q2 only (no memory)", thread_b)


    # B: q3 using thread_id c
    run_verbose(q3, "C-1: q3 only (no memory)", thread_c)

    print("\n" + "#"*10 + " A/B MEMORY TEST END " + "#"*10)


### ㄴ 2.3.2 Test A (같은 thread_id)

*   Q1을 thread_id A 에 저장
*   Q2를 thread_id A 에 질의
----------------------------
| Q2에서 메모리에 저장한 Q1을 활용하여 답변할 것으로 예상

In [ ]:
# 같은 thread_id
run_verbose(q1, "Q1", "chat-001")
run_verbose(q2, "Q2", "chat-001")
run_verbose(q3, "Q3", "chat-001")


==================== Q1 (thread_id=chat-001) ====================
USER: 내 출생년도는 1994년이야. 지금은 계산하지 말고 이 정보만 기억해.

AI: 알겠습니다. 당신의 출생년도는 1994년입니다. 이 정보를 기억하겠습니다.

==================== Q2 (thread_id=chat-001) ====================
USER: 그럼 내가 2026년에 몇 살인지 계산기를 꼭 사용해서 나이를 계산해서 알려줘
tool_calls: [{'name': 'Calculator', 'args': {'__arg1': '2026 - 1994'}, 'id': 'call_6jhnpp81F9wGUN34NPpV01kn', 'type': 'tool_call'}]

TOOL RESULT: Calculator
32

AI: 2026년에 당신은 32살이 됩니다.

==================== Q3 (thread_id=chat-001) ====================
USER: 올해는 무슨 해이며 나는 무슨 띠인지 알려줘
tool_calls: [{'name': 'wikipedia', 'args': {'query': '2023 Chinese zodiac'}, 'id': 'call_AmvWOHGOqU9Z0CRf8Oy81RT7', 'type': 'tool_call'}]

TOOL RESULT: wikipedia
Page: Chinese zodiac
Summary: The Chinese zodiac is a traditional classification scheme based on the Chinese calendar that assigns an animal and its reputed attributes to each year in a repeating twelve-year (or duodenary) cycle. The zodiac is very important in traditional Chi

### ㄴ 2.3.3 TEST B (다른 thread_id)

*   Q1을 thread_id A 에 저장
*   Q2를 thread_id B에 질의
----------------------------
| Q2에서 질의 에 대한 정보 요구 예상



In [ ]:
ab_test(q1, q2, q3, thread_a="chat-003", thread_b="chat-004", thread_c="chat-005")



########## A/B MEMORY TEST START ##########

==================== A-1: seed memory (q1) (thread_id=chat-003) ====================
USER: 내 출생년도는 1994년이야. 지금은 계산하지 말고 이 정보만 기억해.

AI: 알겠습니다. 당신의 출생년도는 1994년입니다. 이 정보를 기억하겠습니다.

==================== B-1: q2 only (no memory) (thread_id=chat-004) ====================
USER: 그럼 내가 2026년에 몇 살인지 계산기를 꼭 사용해서 나이를 계산해서 알려줘
tool_calls: [{'name': 'Calculator', 'args': {'__arg1': '2026 - 2023'}, 'id': 'call_yod3LL1DVXBKgpvpXZTkCOX2', 'type': 'tool_call'}]

TOOL RESULT: Calculator
3

AI: 2026년에는 현재 나이에서 3살을 더한 나이가 됩니다. 현재 나이를 알려주시면 2026년에 몇 살이 되는지 정확히 계산해드릴 수 있습니다.

==================== C-1: q3 only (no memory) (thread_id=chat-005) ====================
USER: 올해는 무슨 해이며 나는 무슨 띠인지 알려줘

AI: 2023년은 음력으로 계묘년(癸卯年)으로, 이는 토끼띠에 해당합니다. 당신의 띠를 알기 위해서는 태어난 해가 필요합니다. 태어난 해를 알려주시면 어떤 띠인지 알려드릴 수 있습니다.

########## A/B MEMORY TEST END ##########


# 3. Search-augmented ReAct (Self-ask with search)

*   질문 분해 중심
*   **RAG와 매우 유사**
*   사실상 Self-Ask = 초기 형태의 RAG



## ㄴ 3.1 Description

### ㄴ 3.1.1. Workflow



```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph / Agent Runtime 시작
  ↓
LLM이 질문을 "하위 질문(Sub-questions)"으로 분해
  ↓
각 하위 질문을 Search Tool로 순차 조회
  ↓
검색 결과(Observation)를 모아 중간 결론 생성
  ↓
필요하면 추가 하위 질문 생성 → 다시 Search
  ↓
모든 정보가 충분해지면 최종 답 생성
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시
```



### ㄴ 3.1.2 Inner ReAct Loop

```javascript
while 답을 만들 정보가 부족하면:
    Follow-up Question 생성
    Search Tool 실행
    Observation 수집
    현재까지의 사실 정리
```

## ㄴ 3.2 Code

In [ ]:
from langchain_core.messages import SystemMessage

# Self-Ask의 핵심 포맷을 강제
SELF_ASK_POLICY = SystemMessage(content="""
You are a Self-Ask-with-Search agent.

Hard rules:
- For EACH follow-up question, you MUST call the tavily_search_results_json tool.
- After the tool returns, you MUST write an Intermediate answer that uses the tool result.
- Intermediate answer must include 2-3 bullet points and mention the source titles (from tool results).
- Do NOT write placeholders like "...". If info is missing, do another search.

Output format (exact):
Follow-up 1: ...
Intermediate answer 1:
- ...
- ...
Sources: <title1>, <title2>

Follow-up 2: ...
Intermediate answer 2:
- ...
- ...
Sources: ...

Final answer: ...
""")


In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

/tmp/ipython-input-2357398768.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


In [ ]:
q = """AI Agent를 3문장으로 설명해줘.
단, 2025~2026년 기준으로 최신 경향을 반영하기 위해 반드시 tavily_search_results_json을 최소 2번 사용하고,
각 검색 결과를 근거로 Intermediate answer를 작성한 뒤 Final answer를 써."""

In [ ]:
config = {"configurable": {"thread_id": "self-ask-01"}}

result = agent.invoke({"messages": [SELF_ASK_POLICY, ("user", q)]},config=config)

print(result["messages"][-1].content)

AI 에이전트는 특정 작업이나 문제를 해결하기 위해 설계된 소프트웨어 프로그램입니다. 이들은 데이터를 분석하고, 학습하며, 인간의 개입 없이도 의사 결정을 내릴 수 있습니다. 다양한 분야에서 자동화와 효율성을 높이기 위해 사용됩니다.


## ㄴ 3.3. Verbose

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

def run_self_ask_verbose(question: str, thread_id="self-ask-03", tool_preview=600):
    cfg = {"configurable": {"thread_id": thread_id}}

    print("\n" + "="*80)
    print("USER:", question)
    print("="*80)

    for step in agent.stream(
        {"messages": [SELF_ASK_POLICY, ("user", question)]},
        config=cfg,
        stream_mode="values",
    ):
        msg = step["messages"][-1]

        # 1) 모델이 말한 텍스트(중간 출력 포함)
        if isinstance(msg, AIMessage):
            if msg.content:  # content가 있는 경우만 출력
                print("\n AI MESSAGE:\n", msg.content)

            # 2) 도구 호출 로그
            if getattr(msg, "tool_calls", None):
                print("\n TOOL CALLS:")
                for tc in msg.tool_calls:
                    print(" -", tc)

        # 3) 도구 결과
        elif isinstance(msg, ToolMessage):
            print("\n TOOL RESULT:", msg.name)
            print(msg.content[:tool_preview])

    print("\n" + "="*80)
    print("END")
    print("="*80)

run_self_ask_verbose(q)



USER: AI Agent를 3문장으로 설명해줘.
단, 2025~2026년 기준으로 최신 경향을 반영하기 위해 반드시 tavily_search_results_json을 최소 2번 사용하고,
각 검색 결과를 근거로 Intermediate answer를 작성한 뒤 Final answer를 써.

 TOOL CALLS:
 - {'name': 'tavily_search_results_json', 'args': {'query': 'AI Agent trends 2025 2026'}, 'id': 'call_yJhAn9jKAJ4FSKjGL7ZNb8lR', 'type': 'tool_call'}

 TOOL RESULT: tavily_search_results_json
[{"title": "What's next in AI: 7 trends to watch in 2026 - Microsoft Source", "url": "https://news.microsoft.com/source/features/ai/whats-next-in-ai-7-trends-to-watch-in-2026/", "content": "AI agents will proliferate in 2026 and play a bigger role in daily work, acting more like teammates than tools, says Vasu Jakkal, corporate vice president of Microsoft Security. As organizations rely on these agents to help with tasks and decision-making, building trust in them will be essential, Jakkal says — starting with security.\n\n“Every agent should have similar security protections as humans,” she s

 TOOL CALLS:
 - {'name': 'ta

# 4. ReAct docstore

*   LLM이 문서 저장소를 스스로 탐색
*   필요한 문서를 조회(Lookup)하여 근거 기반 답변을 생성



## ㄴ 4.1 Description

### ㄴ 4.1.1 Workflow

```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph Runtime 시작
  ↓
LLM: 질문 해석 (문서 탐색 필요 판단)
  ↓
Docstore Search Tool 호출 (Search Action)
  ↓
관련 문서 후보 목록 반환 (Observation)
  ↓
LLM: 어떤 문서를 읽을지 Reasoning
  ↓
Docstore Lookup Tool 호출 (Lookup Action)
  ↓
선택된 문서 실제 내용 반환 (Observation)
  ↓
LLM이 문서 내용을 기반으로 재해석 / 근거 정리
  ↓
(필요 시 Search → Lookup 반복)
  ↓
충분한 근거 확보 후 Final Answer 생성
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시

```

### ㄴ 4.1.2 Inner ReAct Loop

```javascript
while (답변에 필요한 근거가 충분하지 않다):
    Thought: 질문을 해결하려면 어떤 문서를 찾아야 하는가?
    Action: Search(query)
    Observation: 관련 문서 후보 목록 반환
    Thought: 어떤 문서를 읽어야 하는가?
    Action: Lookup(document_id)
    Observation: 문서 내용 확보
    Thought:이 정보로 답변 가능한가?
            ├─ NO → 다른 문서 다시 Search
            └─ YES → 반복 종료
```

## ㄴ 4.2 Code

In [ ]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

/tmp/ipython-input-2357398768.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


In [ ]:
q = """
규칙:
- wikipedia 도구를 1회 호출해서 후보 제목을 찾는다.
- 그 다음 docstore_lookup 도구를 반드시 1회 이상 호출한다.
- docstore_lookup 결과를 근거로 최종 답변을 작성한다.

질문: 밍크 선인장(Mammillaria)은 어떤 식물인지 설명해줘.
"""

In [ ]:

config = {"configurable": {"thread_id": "docstore-01"}}
result = agent.invoke({"messages": [("user", q)]},config=config)

print(result["messages"][-1].content)


Mammillaria는 선인장과(Cactaceae)에 속하는 속으로, 약 200여 종의 종과 변종이 알려져 있습니다. 이 속의 식물들은 주로 멕시코가 원산지이며, 미국 남서부, 카리브해, 콜롬비아, 과테말라, 온두라스, 베네수엘라에서도 발견됩니다. 일반적으로 "바늘방석 선인장"으로 불리며, 이는 이 속과 밀접하게 관련된 Escobaria와도 관련이 있습니다.

Mammillaria는 다양한 형태와 크기를 가지며, 구형 또는 원통형의 줄기를 가지고 있습니다. 이 식물들은 단독으로 자라거나 군집을 이루며, 줄기는 작은 원뿔 모양의 결절로 덮여 있습니다. 결절은 종종 가시나 털로 장식되어 있으며, 이로 인해 독특한 외관을 가지게 됩니다. 꽃은 주로 식물의 상단에서 피며, 흰색, 녹색, 노란색, 분홍색, 빨간색 등 다양한 색상을 띠고 있습니다. 이러한 꽃들은 선인장에 색다른 매력을 더해주며, 벌과 나비 같은 수분 매개체를 유인할 수 있습니다.

Mammillaria는 관리가 쉬운 식물로, 잘 배수되는 토양과 밝지만 직사광선이 아닌 햇빛, 그리고 적절한 물주기를 통해 잘 자랄 수 있습니다. 이 식물들은 독성이 없으며, 사람과 애완동물에게 안전합니다. 그러나 가시는 날카로울 수 있어 주의가 필요합니다.


## ㄴ 4.3 Verbose

In [ ]:
from langchain_core.messages import AIMessage, ToolMessage

def run_docstore_verbose(question: str, thread_id="docstore-debug"):
    cfg = {"configurable": {"thread_id": thread_id}}

    print("\n================ USER ================")
    print(question)

    for step in agent.stream(
        {"messages": [("user", question)]},
        config=cfg,
        stream_mode="values",   # 상태 변화 전부 받기
    ):
        msg = step["messages"][-1]

        # LLM이 생각한 내용 (Thought / Action 결정)
        if isinstance(msg, AIMessage):
            if msg.content:
                print("\n AI THOUGHT:")
                print(msg.content)

            if getattr(msg, "tool_calls", None):
                print("\n TOOL CALL:")
                print(msg.tool_calls)

        # Tool 실행 결과 (Observation)
        elif isinstance(msg, ToolMessage):
            print("\n TOOL RESULT:", msg.name)
            print(msg.content[:1000])  # 너무 길어서 제한

    print("\n================ END ================\n")

run_docstore_verbose(q)



================ USER ================

규칙:
- wikipedia 도구를 1회 호출해서 후보 제목을 찾는다.
- 그 다음 docstore_lookup 도구를 반드시 1회 이상 호출한다.
- docstore_lookup 결과를 근거로 최종 답변을 작성한다.

질문: 밍크 선인장(Mammillaria)은 어떤 식물인지 설명해줘.


 TOOL CALL:
[{'name': 'wikipedia', 'args': {'query': 'Mammillaria'}, 'id': 'call_Mi4xnm2Iq74lHNJtLRSdd0vJ', 'type': 'tool_call'}]

 TOOL RESULT: wikipedia
Page: Mammillaria
Summary: Mammillaria is one of the largest genera in the cactus family (Cactaceae), with currently 200 known species and varieties recognized. Most of the mammillarias are native to Mexico, while some come from the Southwestern United States, the Caribbean, Colombia, Guatemala, Honduras and Venezuela. The common name "pincushion cactus" refers to this genus and the closely related Escobaria.
The first species was described by Carl Linnaeus as Cactus mammillaris in 1753, deriving its name from the Latin mammilla, "nipple", referring to the tubercles that are among the distinctive features of the genus. Numerous spec